# Local Attack Analysis

TJ Kim
11.5.21

#### Summary:
- Load the locally trained model and FedEM model
- nodes = 3, mixtures = 3

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Relevant Libraries
Take it from the run_experiment.py folder

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

### Generate Aggregator Pre-requisite
- Clients, Test Clients, Ensemble_Learner
- Follow through the code in run_experiment.py

In [3]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "local"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 1
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_11_06_local/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 224.83it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:12<00:00,  6.36it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.299 | Train Acc: 10.643% |Test Loss: 2.298 | Test Acc: 10.503% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [4]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

In [5]:
# Set model weights
weights = np.load("weights/cifar/21_11_06_local/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]

# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for i in range(num_models):
    new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
    new_model.eval()
    models_test += [new_model]

### Generate Data 

In [6]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [7]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

### Set Up Transfer Attack Scenario
- Set up order of which attacks will take place -- keep the same transferer, but simply swap out the weights of the adversary and flush out the existing data points analysis was done on
- Make dictionaries beforehand recording all of the metrics

In [8]:
# Set Up Dictionaries -- list holds the adversary idx
logs_adv = []

for i in range(len(model_weights)):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities'] = None
    adv_dict['orig_target_hit'] = None
    adv_dict['adv_target_hit'] = None
    adv_dict['metric_variance'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['metric_ingrad'] = None
    logs_adv += [adv_dict]

In [9]:
# Make transferer and Assign model index
victim_idxs = [0,1,2,3,4,5,6]

for adv_idx in victim_idxs:
    print("id", adv_idx)
    # Perform Attack
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    t1.ifsgm_params.set_params(batch_size=500, eps=0.1, alpha=0.05, iteration = 30,
                   target = 9, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))
    t1.generate_victims(victim_idxs)
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "ifsgm")
    t1.send_to_victims(victim_idxs)
    # t1.check_empirical_metrics(orig_flag = True)
    # t1.check_empirical_metrics()
    
    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = t1.orig_acc_transfers
    logs_adv[adv_idx]['orig_similarities'] = t1.orig_similarities
    logs_adv[adv_idx]['adv_acc_transfers'] = t1.adv_acc_transfers
    logs_adv[adv_idx]['adv_similarities'] = t1.adv_similarities
    logs_adv[adv_idx]['orig_target_hit'] = t1.orig_target_hit
    logs_adv[adv_idx]['adv_target_hit'] = t1.adv_target_hit
    
    logs_adv[adv_idx]['metric_variance'] = t1.metric_variance
    logs_adv[adv_idx]['metric_alignment'] = t1.metric_alignment
    logs_adv[adv_idx]['metric_ingrad'] = t1.metric_ingrad

id 0
id 1
id 2
id 3
id 4
id 5
id 6


### Print and Record Results
- Organize the results into a matrix and print them into an excel sheet
- Following Categories (sim_benign, sim_adv, target_adv,grad_alignment)

In [10]:
metrics = ['orig_acc_transfers','orig_similarities','adv_similarities','adv_target_hit','metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_benign = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
target_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
# grad_align = np.zeros([len(victim_idxs),len(victim_idxs)]) 

In [11]:
for adv_idx in victim_idxs:
    for victim in victim_idxs:
        orig_acc[adv_idx,victim] = logs_adv[adv_idx][metrics[0]][victim].data.tolist()
        sim_benign[adv_idx,victim] = logs_adv[adv_idx][metrics[1]][victim].data.tolist()
        sim_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[2]][victim].data.tolist()
        target_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[3]][victim].data.tolist()
        # grad_align[adv_idx,victim] = logs_adv[adv_idx][metrics[4]][victim].data.tolist()

In [12]:
# Save to Excel file

## convert your array into a dataframe
orig_acc_df = pd.DataFrame(orig_acc)
sim_benign_df = pd.DataFrame(sim_benign)
sim_adv_df = pd.DataFrame(sim_adv)
target_adv_df = pd.DataFrame(target_adv)
# grad_align_df = pd.DataFrame(grad_align)

## save to xlsx file

#filepath = 'my_excel_file.xlsx'

# df.to_excel(filepath, index=False)

In [13]:
orig_acc_df

,0,1,2,3,4,5,6
0,0.358,0.368,0.508,0.404,0.430,0.544,0.454
1,0.332,0.324,0.476,0.388,0.440,0.550,0.404
2,0.354,0.336,0.446,0.396,0.444,0.504,0.442
3,0.374,0.354,0.446,0.402,0.466,0.576,0.452
4,0.384,0.328,0.454,0.366,0.450,0.576,0.462
5,0.324,0.370,0.450,0.402,0.460,0.496,0.402
6,0.360,0.340,0.480,0.424,0.424,0.540,0.436


In [14]:
sim_benign_df

,0,1,2,3,4,5,6
0,1.000,0.312,0.366,0.228,0.226,0.418,0.384
1,0.288,1.000,0.382,0.320,0.224,0.370,0.410
2,0.368,0.440,1.000,0.344,0.298,0.472,0.452
3,0.232,0.368,0.374,1.000,0.354,0.316,0.302
4,0.240,0.216,0.334,0.350,1.000,0.338,0.256
5,0.412,0.422,0.482,0.308,0.284,1.000,0.468
6,0.386,0.440,0.468,0.322,0.270,0.512,1.000


In [15]:
sim_adv_df

,0,1,2,3,4,5,6
0,1.000,0.260,0.244,0.174,0.096,0.252,0.250
1,0.204,1.000,0.224,0.256,0.112,0.268,0.290
2,0.158,0.290,1.000,0.186,0.062,0.228,0.252
3,0.188,0.284,0.276,1.000,0.206,0.250,0.234
4,0.196,0.296,0.238,0.246,1.000,0.228,0.210
5,0.226,0.330,0.220,0.154,0.068,1.000,0.268
6,0.192,0.314,0.236,0.214,0.058,0.218,1.000


In [16]:
target_adv_df

,0,1,2,3,4,5,6
0,0.908,0.254,0.226,0.174,0.066,0.212,0.224
1,0.156,0.878,0.162,0.192,0.050,0.214,0.232
2,0.142,0.270,0.938,0.162,0.036,0.204,0.228
3,0.164,0.254,0.222,0.748,0.048,0.198,0.212
4,0.200,0.278,0.214,0.196,0.728,0.206,0.216
5,0.222,0.324,0.214,0.148,0.066,0.988,0.262
6,0.184,0.308,0.228,0.208,0.052,0.212,0.986


In [17]:
# grad_align_df

In [18]:
# Average all the information together and present
orig_acc_mean = np.mean(orig_acc)
sim_benign_mean = np.mean((sim_benign.sum(1)-1)/(sim_benign.shape[1]-1))
sim_adv_mean = np.mean((sim_adv_df.sum(1)-1)/(sim_adv_df.shape[1]-1))
target_adv_mean = np.mean((target_adv.sum(1)-1)/(target_adv.shape[1]-1))
# grad_align_mean = np.mean((grad_align.sum(1)-0)/(grad_align.shape[1]-0))

print("orig_acc_mean", orig_acc_mean)
print("Sim Benign Mean", sim_benign_mean)
print("Sim ADV Mean", sim_adv_mean)
print("Target ADV Mean", target_adv_mean)
# print("Grad Align Mean", grad_align_mean)

orig_acc_mean 0.42653062878822795
Sim Benign Mean 0.35133335136231925
Sim ADV Mean 0.21800000965595245
Target ADV Mean 0.17176192040954316
